# Estimating functional connectivity on ABIDE dataset

First, we download the preprocessed time series extracted using Craddock 200 atlas from Preprocessed Connectome Project.

In [3]:
from nilearn.datasets.func import fetch_abide_pcp


abide = fetch_abide_pcp(n_subjects=None, quality_checked=True,
                        band_pass_filtering=True, pipeline='niak',
                        derivatives='rois_cc200')
abide.keys()

dict_keys(['description', 'rois_cc200', 'phenotypic'])

We compute the connectivity matric for each of the subjects. We use the "tangent" method provided by nilearn.

In [5]:
from nilearn.connectome import ConnectivityMeasure


conn_est = ConnectivityMeasure(kind="tangent")
conn_matrices = conn_est.fit_transform(abide.rois_cc200)
conn_matrices.shape

(871, 200, 200)

We have connectivity matrices but we only want the values for the lower (or upper) triangular part. Fortunately, a function in nilearn does that for free.

In [7]:
from nilearn.connectome import sym_to_vec


X = sym_to_vec(conn_matrices)
X.shape

(871, 20100)

## Preparing cross-validation

The cross-validation scheme is very important. Here we chose to take 20 balanced folds with randomly picked subjects.

In [8]:
from sklearn.cross_validation import StratifiedShuffleSplit


id = [str(sid) + str(dx) for sid, dx in abide.phenotypic[['SITE_ID', 'DX_GROUP']]]
cv = StratifiedShuffleSplit(id, n_iter=20, test_size=.2)

## Prediction

We use a classifier to diagnose autistic vs. healthy participants.

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.cross_validation import cross_val_score


y = abide.phenotypic['DX_GROUP']
pred = LinearSVC(C=0.01)
np.mean(cross_val_score(pred, X, y, cv=cv))